# python调用C/C++程序
非原创

这里只讨论两种简单易用的Python调用C/C++程序的方法：dll调用和C扩展调用。
### DLL 调用
(1) 首先编写C代码程序文件libtest.c如下：
```c
// Linux
int multiply(int num1, int num2){
return num1 * num2;}

// Windows
#include <windows.h>
BOOL APIENTRY
DllMain(HANDLE hModule, DWORD dwReason, LPVOID lpReserved){
	return TRUE;}
__declspec(dllexport) int multiply(int num1, int num2){
	return num1 * num2;}
```
(2) 然后将该源码文件编译为动态链接库
```shell
// Linux
gcc -c -fPIC libtest.c
gcc -shared libtest.o -o libtest.so

// Windows
cl -LD libtest.c
```
(3) 得到动态链接库文件以后就可以直接在Python中进行调用。
```python
from ctypes import *
import os
// Windows-> libtest.dll
libtest = cdll.LoadLibrary(os.getcwd() + '/libtest.so')
print libtest.multiply(2, 2) >> 4
```

DLL的调用方式比较简单，对原先代码几乎可以不加修改，但是该方式对传入和传出的参数类型有较多限制，在多个参数传递时需要进行参数结构的定义，应用比较麻烦。

### C扩展
(1) 首先编写C代码程序文件CExtest.c如下：
```c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include "Python.h"
#define BUFSIZE 10

char *reverse(char *s) {    
	register char t;
	char *p = s; 
	char *q = (s + (strlen(s) - 1));    
	while (p < q) {       
		t = *p;      
		*p++ = *q;       
		*q-- = t;    
	}    
	return s;
}

int main() {    
	char s[BUFSIZE];    
	strcpy(s, "abcdef");    
	printf("reversing 'abcdef', we get '%s'\n", reverse(s));    
	strcpy(s, "madam");    
	printf("reversing 'madam', we get '%s'\n", reverse(s));    
	return 0;
}

static PyObject *Extest_reverse(PyObject *self, PyObject *args) {    
	char *orignal;    
	//s表示需要传递进来的参数类型为字符串，
	//如果是，就赋值给original，如果不是，返回NULL；
	if (!(PyArg_ParseTuple(args, "s", &orignal))) {        
	//包装函数返回NULL，就会在Python调用中产生一个TypeError的异常
	return NULL;}    
	//需要把c中计算的结果转成python对象，s代表字符串对象类型。
	return (PyObject *)Py_BuildValue("s", reverse(orignal));
}

static PyMethodDef ExtestMethods[] = {    
//{"fac", Extest_fac, METH_VARARGS},    
//{"doppel", Extest_doppel, METH_VARARGS},    
{"reverse", Extest_reverse, METH_VARARGS},  
{NULL, NULL},
};

void initExtest() {    
	Py_InitModule("Extest", ExtestMethods);}
```
(2) 编写setup.py文件
```python
# encoding: utf-8
from distutils.core import setup, Extension
MOD = 'Extest'
setup(	name=MOD,
	ext_modules=[
		Extension(
			MOD, 
			sources=['CExtest.c'])])
```
(3) 开始编译，在该目录下运行 python setup.py build\_ext   - -inplace\\
Windows下可能会报错：error: Unable to find vcvarsall.bat\\
报错原因是没有安装支持的编译器，可以从<https://www.microsoft.com/en-us/download/details.aspx?id=44266>下载支持Python2.7的编译器VC9.0。\\
(4) Windows下，安装完成后需要修改注册表添加项\\
32位：
```
HKEY_CURRENT_USER\Software\Microsoft\VisualStudio\9.0\Setup\VC
```
64位：
```
HKEY_CURRENT_USER\Software\Wow6432Node\Microsoft\VisualStudio\9.0\Setup\VC
```
还需要在VC项下新建字符串\\
名称：productdir\\
数据：vcvarsall.bat所在路径\\
注意：默认该文件在C盘中，直接搜索即可，路径中不包含最后的反斜杠。\\
(5) 再次运行(3)中的代码进行编译，在当前目录下会出现一个CExtest.pyd文件，可以直接在Python中导入，如果还是报头文件和lib文件找不到的错误就在环境变量中添加INCLUDE和LIB两个，把对应的路径添加进去即可。
```Python
import CExtest
print CExtest.reverse('hello')>>'olleh'
```

C扩展的方式本质上也是动态链接库，只是对函数的调用接口进行了Python的标准化处理，因此程序调用跟Python本身的一致，比较方便。更详细的过程可以参阅<http://www.jianshu.com/p/136c9912e929>,但这个教程最后处理Python接口的代码中有错误，须将不存在的接口函数注释掉。